In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import mailbox

In [3]:

mailboxfile = "C:/Users/HP/takeout-20220527T154953Z-001/Takeout/Mail/All mail Including Spam and Trash.mbox"

mbox = mailbox.mbox(mailboxfile)
mbox

In [4]:
for key in mbox[0].keys():
    print(key)

X-GM-THRID
X-Gmail-Labels
Delivered-To
Received
X-Received
ARC-Seal
ARC-Message-Signature
ARC-Authentication-Results
Return-Path
Received
Received-SPF
Authentication-Results
DKIM-Signature
X-Google-DKIM-Signature
X-Gm-Message-State
X-Google-Smtp-Source
MIME-Version
X-Received
Date
List-Unsubscribe
X-Notifications
X-Notifications-Bounce-Info
Message-ID
Subject
From
To
Content-Type


In [5]:
import csv

In [ ]:
with open('mailbox.csv', 'w') as outputfile:
    writer = csv.writer(outputfile)
    writer.writerow(['subject','from','date','to','label','thread'])
    for message in mbox:
        writer.writerow([
            message['subject'],
            message['from'],
            message['date'],
            message['to'],
            message['X-Gmail-Labels'],
            message['X-GM-THRID']
        ])
        

In [ ]:
dfs = pd.read_csv('mailbox.csv', names=['subject', 'from', 'date', 'to',
'label', 'thread'])

In [ ]:
dfs.head(5)

In [ ]:
dfs.dtypes

In [ ]:
dfs['date'] = dfs['date'].apply(lambda x: pd.to_datetime(x, errors='coerce', utc=True))

In [ ]:
dfs.dtypes

In [ ]:
## Removing the Nan values

In [ ]:
dfs = dfs[dfs['date'].notna()]

In [ ]:
## Applying descriptive statistics

In [ ]:
dfs.info()

In [ ]:
# Note that ther are 21565 emails

In [ ]:
dfs.head(10)

In [ ]:
## Data refactoring

In [ ]:
# creating an email extractor

import re

def extract_email_address(string):
    email = re.findall(r'<(.+?)>', string)
    if not email:
        email = list(filter(lambda y: '@' in y, string.split()))
        return email[0] if email else np.nan


In [ ]:
# applying the email extractor

dfs['from'] = dfs['from'].apply(lambda x: extract_email_address(x))

In [ ]:
dfs.sample(10)

In [ ]:
myemail = 'ayosamobitade@gmail.com'
dfs['label'] = dfs['from'].apply(lambda x: 'sent' if x==myemail else 'inbox')

## Dropping columns


In [ ]:
dfs.drop('to', axis = 1, inplace = True)

In [ ]:
# adjusting the time zones

import datetime
import pytz

def refaactor_timezone(x):
    eas = pytz